# Setup

In [ ]:
from dataclasses import dataclass

import pandapower.networks as pnet

import numpy as np
import cvxpy as cp
from numpy.linalg import inv

In [ ]:
import sys
sys.path.insert(1, '..')

from src.models.regression import ComplexRegression, ComplexLasso
from src.simulation.load_profile import generate_gaussian_load
from src.simulation.network import add_load_power_control, make_y_bus
from src.simulation.simulation import run_simulation, get_current_and_voltage
from src.identification.error_metrics import error_metrics, fro_error

# Network simulation
To start, we simulate Gaussian and independent fluctuations for power demands of loads, close to their referece values.

In [ ]:
net = pnet.case6ww()
nodes = net.bus.shape[0]
samples = 200
load_cv = 0.02

In [ ]:
load_p, load_q = generate_gaussian_load(net.load.p_mw, net.load.q_mvar, load_cv, samples)
controlled_net = add_load_power_control(net, load_p, load_q)
sim_result = run_simulation(controlled_net, verbose=False)
y_bus = make_y_bus(controlled_net)
voltage, current = get_current_and_voltage(sim_result, y_bus)

# OLS Identification

In [ ]:
y_ols = ComplexRegression().fit(voltage, current).fitted_beta
error_metrics(y_bus, y_ols)

# Lasso Identification

TODO: explain notation.

In order to apply the complex LASSO algirithm, we first need to perform a vectorization of our model, as in (Ardakanian, 2019): 
\begin{equation}
\text{vect}(I) = \text{vect}(VY) = \left( \mathbb{I}_n \otimes V \right) \text{vect} (Y).
\end{equation}

Thus, the optimization problem we need to solve is:
\begin{equation}
\hat x = \arg \min_x \left \Vert b - Ax \right \Vert^2_2 + \lambda \left \Vert x \right \Vert_1.
\end{equation}

where:
\begin{align}
A&= \text{vect}{(\underline{V}^T)} \otimes \mathbb{I}_n, \\
b &= \text{vect} (\underline{I}), \\
x &= \text{vect} (Y),
\end{align}

In the following, inputs and outputs of the model will be called y and x, respectively.

In [ ]:
b = np.ravel(current, 'F')
a = np.kron(np.eye(nodes), voltage)

Then, we can apply the r-LASSO, defined in (Maleki, 2013), following what done by (Ardakanian, 2019) - see https://github.com/sustainable-computing/Distribution-Grid-Identification/blob/master/standardLasso.m.

In order to solve the problem, we frame it as an uncontrained convex optimization problem, using the Python cvxpy library: https://github.com/cvxgrp/cvxpy.

In [ ]:
lasso_res = ComplexLasso(verbose=False, lambdas=np.logspace(-10, 1, 100)).fit(a, b)
best_lasso_res = lasso_res.get_best_by(lambda y: fro_error(y.reshape(nodes, nodes), y_bus))

In [ ]:
best_lasso_res.lambda_value

In [ ]:
best_lasso_res.metric_value